1. Check synchronization between image arrays - 
2. Check synchronization between labels and hires image arrays - Done
3. Downsampling no tool frames and getting rid of false label frames

3.5 Data Augmentation
4. Create masks using FCN 
5. Dynamic Zoom 

Make array copy efficient. Use indexing instead of vstack

In [1]:
import imageio
imageio.plugins.ffmpeg.download()
import moviepy.editor as mpe
from glob import glob
import sys, os
import numpy as np
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from scipy import sparse
from sklearn.utils.extmath import randomized_svd
import bcolz
import utils; reload(utils)
from utils import *
import scipy.signal as sci
from keras.preprocessing.image import ImageDataGenerator
import itertools


Imageio: 'ffmpeg.linux64' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64 (27.2 MB)
Downloading: 8192/28549024 bytes (0.0%)901120/28549024 bytes (3.2%)2555904/28549024 bytes (9.0%)5087232/28549024 bytes (17.8%)8216576/28549024 bytes (28.8%)11255808/28549024 bytes (39.4%)14368768/28549024 bytes (50.3%)17481728/28549024 bytes (61.2%)20594688/28549024 bytes (72.1%)23707648/28549024 bytes (83.0%)26828800/28549024 bytes (94.0%)28549024/28549024 bytes (100.0%)
  Done
File saved as /home/ubuntu/.imageio/ffmpeg/ffmpeg.linux64.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
def create_data_matrix_from_video(clip,totalFrames,fps):
    return np.array([frame for frame,i in itertools.ifilter(lambda x: x[1]%fps==0,
                                                            zip(clip.iter_frames(),range(totalFrames)))])

In [3]:
def create_labels_for_video(path,name,totalFrames,fps):
    p = pd.read_csv(path + name + '.csv')
    return np.array([row for row,i in itertools.ifilter(lambda x: x[1]%fps==0,
                                                            zip(p.itertuples(),range(totalFrames)))])   

In [4]:
def augmentData(mrgb):
    #Need to add  code to simultaneously augment all arrays together
    
    hflip = ImageDataGenerator(horizontal_flip=True)
    vflip = ImageDataGenerator(vertical_flip=True)
    rotate = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
    zoom = ImageDataGenerator(zoom_range=(0.8,1.1))

    totalFrames = mrgb.shape[0]
    mrgbAug = np.zeros([totalFrames, 3, 360, 640 ],dtype=np.uint8)

    for frameIndex in range(totalFrames):
        x_train = np.transpose(mrgb[frameIndex],(2,0,1))
        images_0 = np.zeros([1, 3, 360, 640 ],dtype=np.uint8)
        labels_dummy = np.zeros([1, 1],dtype=np.int8)
        images_0[0] = x_train
        images = np.zeros([1, 3, 360, 640 ],dtype=np.uint8)
        if frameIndex%5==0:
            for x,y in hflip.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==1:
            for x,y in vflip.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==2:
            for x,y in rotate.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==3:
            for x,y in zoom.flow(images_0,labels_dummy,batch_size=1):
                mrgbAug[frameIndex] = x
                break
        if frameIndex%5==4:
            mrgbAug[frameIndex] = x_train
    del(mrgb)
    gc.collect()
    return mrgbAug

In [5]:
def processVideos(path,name):
    
    k=5
    video = mpe.VideoFileClip(path+name+'.mp4',target_resolution=(360,640))
    totalFrames = len([1 for frame in video.iter_frames()])
    fps=int(math.ceil(video.fps/k))
    
    RGB_HiRes = augmentData(create_data_matrix_from_video(video, totalFrames,fps))
    labels = create_labels_for_video(path,name,totalFrames,fps)
    
    save_array('../03_output/'+name+'_rgb_hires.dat',RGB_HiRes)
    save_array('../03_output/'+name+'_labels.dat',labels)
    
    del(RGB_HiRes)
    del(labels)
    gc.collect()

In [6]:
videos = glob('../01_input/train/*.mp4')   
totalDuration = 0
for i in range(len(videos)):
    name = videos[i].split("../01_input/train/")[1].split('.mp4')[0]
    video = mpe.VideoFileClip('../01_input/train/'+name+'.mp4')
    print video.duration, video.size, video.fps, name
    totalDuration += video.duration
print totalDuration

492.8 [1920, 1080] 29.18 train01
426.01 [1920, 1080] 29.18 train17
2434.16 [1920, 1080] 29.19 train19
739.11 [1920, 1080] 29.18 train12
940.93 [1920, 1080] 29.18 train08
437.93 [1920, 1080] 29.18 train05
876.18 [1920, 1080] 29.18 train10
474.28 [1920, 1080] 29.2 train06
480.45 [1920, 1080] 29.18 train13
478.01 [1920, 1080] 29.18 train18
529.72 [1920, 1080] 29.19 train21
1398.14 [1920, 1080] 29.19 train25
624.76 [1920, 1080] 29.18 train16
580.12 [1920, 1080] 29.2 train20
707.93 [1920, 1080] 29.19 train22
547.14 [1920, 1080] 29.19 train09
488.38 [1920, 1080] 29.18 train11
383.05 [1920, 1080] 29.19 train23
599.29 [1920, 1080] 29.18 train03
637.03 [1920, 1080] 29.18 train24
703.84 [1920, 1080] 29.19 train14
471.53 [1920, 1080] 29.18 train15
454.52 [1920, 1080] 29.18 train04
642.82 [1920, 1080] 29.19 train07
405.45 [1920, 1080] 29.19 train02
16953.58


In [7]:
files = ['train01','train02','train03','train04','train05','train06','train07','train08','train09','train10','train11',
         'train12','train13','train14','train15','train16','train17','train18','train20','train21','train22',
         'train23','train24'] # Excluding 'train25' & 'train19' as they are too large.
for i in tqdm(range(len(files))):
    processVideos('../01_input/train/',files[i])

100%|██████████| 23/23 [1:43:32<00:00, 265.37s/it]


In [6]:
files = ['train25'] # Execute on 64 GB aws 'train25' as it is too large.
for i in tqdm(range(len(files))):
    processVideos('../01_input/train/',files[i])

100%|██████████| 1/1 [10:48<00:00, 648.08s/it]


In [ ]:
files = ['train19'] # Execute on 64 GB aws 'train19' as it is too large.
for i in tqdm(range(len(files))):
    processVideos('../01_input/train/',files[i])

One time fix to add one label to train22

In [3]:
temp = load_array('/home/ubuntu/cataract4/03_output/train22_fcf_feat.dat')
temp.shape

(3445, 23, 40, 1024)

In [2]:
labels_val = load_array('/cataract/home/ubuntu/cataract/03_output/train22_labels.dat')
labels_val.shape
temp_labels = np.zeros([3445, 23],dtype=np.float64)
temp_labels[1:3445,0:23]=labels_val[0:3444,0:23]
temp_labels.shape

(3445, 23)

In [4]:
save_array('/cataract/home/ubuntu/cataract/03_output/train22_labels.dat',temp_labels)

# Scrapbook

In [ ]:
processVideos('../01_input/train/','train01')

In [ ]:
#path='../01_input/train/'
#name='train01'
#k=5
#video = mpe.VideoFileClip(path+name+'.mp4',target_resolution=(360,640))
#totalFrames = len([1 for frame in video.iter_frames()])
#fps=int(math.ceil(video.fps/k))

#RGB_HiRes = create_data_matrix_from_video(video, totalFrames,fps)
RGB_HiRes_aug = augmentData(RGB_HiRes)

In [ ]:
RGB_HiRes.shape

In [ ]:
RGB_HiRes_aug.shape

In [ ]:
rgb_hires = load_array('../03_output/train01_rgb_hires.dat')
rgb_hires.shape

In [ ]:
labels = load_array('../03_output/train01_labels.dat')
labels.shape

In [ ]:
temp = np.transpose(rgb_hires,(0,2,3,1))

In [ ]:
plt.imshow(RGB_HiRes[21])

In [ ]:
labels[21]

In [ ]:
plt.imshow(temp[12])

In [ ]:
plt.imshow(temp[7])

In [ ]:
plt.imshow(temp[8])

In [ ]:
plt.imshow(temp[9])

In [ ]:
labels[:,8]

In [ ]:
p = pd.read_csv('../01_input/train/train01.csv')

In [ ]:
p